# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [66]:
from pyspark.sql import SparkSession

In [67]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [68]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [69]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [70]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [71]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [72]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [73]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [74]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating") # the last three parameters are the most essentials 
model = als.fit(training)

Now let's see hwo the model performed!

In [75]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [76]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28|  2.4178493|
|      2|   2.0|     1|  2.5180314|
|      3|   1.0|    13|  0.5200279|
|      4|   2.0|    13|  1.0165759|
|      2|   3.0|     6| 0.39500517|
|      0|   1.0|     3|  1.0162702|
|      2|   1.0|     3| -1.5754822|
|      0|   1.0|    20|   2.115519|
|      1|   1.0|    19|  1.2507565|
|      2|   1.0|    15|   1.720846|
|      2|   1.0|    17|  2.5549703|
|      5|   1.0|     8| 0.62372077|
|      4|   1.0|    23| -1.6796205|
|      2|   1.0|    25|   4.808169|
|      4|   1.0|    29|-0.54862785|
|      3|   1.0|    21|  1.5434537|
|      0|   1.0|    11|  0.9289906|
|      3|   3.0|    14| 0.48203188|
|      4|   3.0|     2|    3.46386|
|      5|   2.0|     0| 0.08129406|
+-------+------+------+-----------+
only showing top 20 rows



In [77]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.890360897122207


The RMSE described our error in terms of the stars rating column.

# WE CAN TRY TO IMPROVE BY DOING THIS 

In [78]:
from pyspark.sql.functions import col, when

In [79]:
predictions = predictions.withColumn("prediction", when(col("prediction").cast("int").cast("float") >= 0, col("prediction").cast("int").cast("float")).otherwise(0))
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|   1.0|    28|       2.0|
|     19|   3.0|    28|       0.0|
|     20|   1.0|    28|       2.0|
|     23|   3.0|    28|       0.0|
|     33|   1.0|    28|       1.0|
|     45|   1.0|    28|       2.0|
|     49|   4.0|    28|       2.0|
|     50|   1.0|    28|       0.0|
|     52|   1.0|    28|       3.0|
|     59|   1.0|    28|       2.0|
|     65|   1.0|    28|       0.0|
|     92|   5.0|    28|       3.0|
|     10|   1.0|    27|       0.0|
|     21|   3.0|    26|       1.0|
|     22|   2.0|    27|       2.0|
|     23|   5.0|    26|       0.0|
|     25|   1.0|    27|       0.0|
|     33|   3.0|    27|       1.0|
|     40|   1.0|    27|       0.0|
|     52|   1.0|    26|       1.0|
+-------+------+------+----------+
only showing top 20 rows



In [80]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.7733405882980469


# <span style="color:pink"> WE CAN SEE THAT CHANGING SMALL THINGS IN THE RESULT IMRPOVE THE EFFECIENTY OF THE MODEL !</span>

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [81]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [82]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     12|    11|
|     21|    11|
|     22|    11|
|     40|    11|
|     59|    11|
|     62|    11|
|     69|    11|
|     72|    11|
|     75|    11|
+-------+------+



In [83]:
reccomendations = model.transform(single_user)

In [84]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     69|    11| 3.9150949|
|     72|    11|  2.470568|
|     75|    11| 1.8536415|
|     12|    11| 1.5664023|
|     40|    11| 1.4294059|
|     21|    11| 1.0567698|
|      0|    11| 0.9289906|
|     22|    11|0.35337994|
|     59|    11|0.15427412|
|     62|    11|-1.5080595|
+-------+------+----------+



We will improve it by adding this line of code

In [85]:
reccomendations = reccomendations.withColumn("prediction", when(col("prediction").cast("int").cast("float") >= 0, col("prediction").cast("int").cast("float")).otherwise(0))
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     69|    11|       3.0|
|     72|    11|       2.0|
|     12|    11|       1.0|
|     21|    11|       1.0|
|     40|    11|       1.0|
|     75|    11|       1.0|
|      0|    11|       0.0|
|     22|    11|       0.0|
|     59|    11|       0.0|
|     62|    11|       0.0|
+-------+------+----------+



# Great Job!